# Video Super Resolution with OpenVINO™
Super Resolution is the process of enhancing the quality of an image by increasing the pixel count using deep learning. This notebook applies Single Image Super Resolution (SISR) to frames in a 360p (480×360) video in 360p resolution. A model called [single-image-super-resolution-1032](https://docs.openvino.ai/2024/omz_models_model_single_image_super_resolution_1032.html), which is available in Open Model Zoo, is used in this tutorial. It is based on the research paper cited below. 

Y. Liu et al., ["An Attention-Based Approach for Single Image Super Resolution,"](https://arxiv.org/abs/1807.06779) 2018 24th International Conference on Pattern Recognition (ICPR), 2018, pp. 2777-2784, doi: 10.1109/ICPR.2018.8545760.

> **NOTE**: The Single Image Super Resolution (SISR) model used in this demo is not optimized for a video. Results may vary depending on the video.


#### Table of contents:

- [Preparation](#Preparation)
    - [Install requirements](#Install-requirements)
    - [Imports](#Imports)
    - [Settings](#Settings)
        - [Select inference device](#Select-inference-device)
    - [Functions](#Functions)
- [Load the Superresolution Model](#Load-the-Superresolution-Model)
- [Superresolution on Video](#Superresolution-on-Video)
    - [Settings](#Settings)
    - [Download and Prepare Video](#Download-and-Prepare-Video)
    - [Do Inference](#Do-Inference)
    - [Show Side-by-Side Video of Bicubic and Superresolution Version](#Show-Side-by-Side-Video-of-Bicubic-and-Superresolution-Version)



## Preparation
[back to top ⬆️](#Table-of-contents:)

### Install requirements
[back to top ⬆️](#Table-of-contents:)


In [1]:
%pip install -q "openvino==2023.3.0"
%pip install -q opencv-python
%pip install -q "pytube>=12.1.0"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Imports
[back to top ⬆️](#Table-of-contents:)


In [1]:
import time
from pathlib import Path

import cv2
import numpy as np
from IPython.display import (
    HTML,
    FileLink,
    Pretty,
    ProgressBar,
    Video,
    clear_output,
    display,
)
import openvino as ov
from pytube import YouTube

In [2]:
# Define a download file helper function
def download_file(url: str, path: Path) -> None:
    """Download file."""
    import urllib.request
    path.parent.mkdir(parents=True, exist_ok=True)
    urllib.request.urlretrieve(url, path)

### Settings
[back to top ⬆️](#Table-of-contents:)


#### Select inference device
[back to top ⬆️](#Table-of-contents:)

select device from dropdown list for running inference using OpenVINO

In [3]:
import ipywidgets as widgets

core = ov.Core()
device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)

device

Dropdown(description='Device:', index=9, options=('CPU', 'GPU.0', 'GPU.1', 'GPU.2', 'GPU.3', 'GPU.4', 'GPU.5',…

In [4]:
# 1032: 4x superresolution, 1033: 3x superresolution
model_name = 'single-image-super-resolution-1032'

base_model_dir = Path('./model').expanduser()

model_xml_name = f'{model_name}.xml'
model_bin_name = f'{model_name}.bin'

model_xml_path = base_model_dir / model_xml_name
model_bin_path = base_model_dir / model_bin_name

if not model_xml_path.exists():
    base_url = f'https://storage.openvinotoolkit.org/repositories/open_model_zoo/2023.0/models_bin/1/{model_name}/FP16/'
    model_xml_url = base_url + model_xml_name
    model_bin_url = base_url + model_bin_name

    download_file(model_xml_url, model_xml_path)
    download_file(model_bin_url, model_bin_path)
else:
    print(f'{model_name} already downloaded to {base_model_dir}')

single-image-super-resolution-1032 already downloaded to model


### Functions
[back to top ⬆️](#Table-of-contents:)


In [5]:
def convert_result_to_image(result) -> np.ndarray:
    """
    Convert network result of floating point numbers to image with integer
    values from 0-255. Values outside this range are clipped to 0 and 255.

    :param result: a single superresolution network result in N,C,H,W shape
    """
    result = result.squeeze(0).transpose(1, 2, 0)
    result *= 255
    result[result < 0] = 0
    result[result > 255] = 255
    result = result.astype(np.uint8)
    return result

## Load the Superresolution Model
[back to top ⬆️](#Table-of-contents:)

Load the model in OpenVINO Runtime with `core.read_model` and compile it for the specified device with `core.compile_model`.

In [5]:
core = ov.Core()
model = core.read_model(model=model_xml_path)
compiled_model = core.compile_model(model=model, device_name=device.value)

Get information about network inputs and outputs. The Super Resolution model expects two inputs: the input image and a bicubic interpolation of the input image to the target size of 1920x1080. It returns the super resolution version of the image in 1920x1080.

In [6]:
# Network inputs and outputs are dictionaries. Get the keys for the
# dictionaries.
original_image_key, bicubic_image_key = compiled_model.inputs
output_key = compiled_model.output(0)

# Get the expected input and target shape. The `.dims[2:]` function returns the height
# and width.The `resize` function of  OpenCV expects the shape as (width, height),
# so reverse the shape with `[::-1]` and convert it to a tuple.
input_height, input_width = list(original_image_key.shape)[2:]
target_height, target_width = list(bicubic_image_key.shape)[2:]

upsample_factor = int(target_height / input_height)

print(f"The network expects inputs with a width of {input_width}, " f"height of {input_height}")
print(f"The network returns images with a width of {target_width}, " f"height of {target_height}")

print(
    f"The image sides are upsampled by a factor of {upsample_factor}. "
    f"The new image is {upsample_factor**2} times as large as the "
    "original image"
)

The network expects inputs with a width of 480, height of 270
The network returns images with a width of 1920, height of 1080
The image sides are upsampled by a factor of 4. The new image is 16 times as large as the original image


In [7]:
image = cv2.cvtColor(cv2.imread('coco_tulips.jpg'), cv2.COLOR_BGR2RGB)
input_image_resized = cv2.resize(image, (input_width, input_height))
input_bicubic_resized = cv2.resize(image, (target_width, target_height), interpolation=cv2.INTER_CUBIC)

input_image_original = np.expand_dims(input_image_resized.transpose(2, 0, 1), axis=0)
input_image_bicubic = np.expand_dims(input_bicubic_resized.transpose(2, 0, 1), axis=0)

openvino_result = compiled_model(
    {
        original_image_key.any_name: input_image_original,
        bicubic_image_key.any_name: input_image_bicubic,
    }
)[output_key]


# PySDK version

In [10]:
import cv2
import degirum as dg
import degirum_tools as dgtools

from superres_postprocessor import SuperResolutionResults

hw_location = dg.LOCAL
zoo_path = 'https://cs.degirum.com/degirum/super_resolution'

model_name = 'singleimage_superresolution_4x--480x270_float_openvino_cpu_1'
# SuperResolutionResults.resize_factor = 4    # Disable postprocessor to compare results directly.

# Alternative model to use for super resolution.
# model_name = 'singleimage_superresolution_3x--640x360_float_openvino_cpu_1'
# SuperResolutionResults.resize_factor = 3

zoo = dg.connect(hw_location, zoo_path, dgtools.get_token())
model = zoo.load_model(model_name, image_backend='opencv', input_image_format="RAW")

pysdk_results = model(['coco_tulips.jpg', 'coco_tulips.jpg'])

In [13]:
# Confirm every element is the same by subtracting elementwise and compare to zero.
np.all(pysdk_results.results[0]['data'] - openvino_result == 0)

True